In [19]:
# ** = Change location if needed
# Weight system is needed: on tags and title and for LDA to pass to LSI

In [18]:
# Seting up the environment:
# Reunite all the text file with the news in a folder called 'Analysis'
# Set up the basic path

basic_path = '/root/Desktop/' # **
path = basic_path + 'Analysis'

folders = ['/Il Corriere della Sera'] 
nick_names = ['/Corr']


choose = 0 # chose the number that represent the journal

actual_journal = [path +  folders[choose], nick_names[choose]]
print(actual_journal)

['/root/Desktop/Analysis/Il Corriere della Sera', '/Corr']


In [20]:
# Complete Analysis
# -----------------------------
# Import library

import re
import ast
import json
import nltk
import time
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [44]:
# Data loading

data = []
with open(actual_journal[0] + '.txt') as f:
    for line in f:
        data.append(json.loads(line, encoding='utf8'))
        print(line)

        
collection_testi = []
for i in range(0, len(data)):
    if len(str(data[i]['text'])) > 10:  # Delete the news with no text
        collection_testi.append(str(data[i]['title']) + str(data[i]['text']) + str(data[i]['tags']) + str(data[i]['resume']))
        
    

{"_id":{"$oid":"5a101c5d5fbf0004f5a4adc4"},"title":["\n                ","\n            "," Di Matteo: mio figlio finito nell’acido, Riina doveva morire 50 anni fa"],"text":"\n ROMA «È morto troppo tardi, doveva morire cinquant’anni fa», mormora nel suo incorreggibile dialetto siciliano Santino Di Matteo, il pentito di mafia che ha confessato la strage di Capaci e al quale Cosa nostra, dopo le sue prime dichiarazioni ai magistrati, ha rapito il figlio tredicenne Giuseppe, ucciso dopo due anni di segregazione. Un ex mafioso, colpevole di tanti omicidi, a sua volta vittima della mafia che adesso dice: «Sono contento che è morto Totò Riina, così finalmente si chiude un capitolo».   «Certo, perché noi l’abbiamo seguito e abbiamo sbagliato. Ci siamo fidati delle famiglie che gli stavano intorno, come i Madonia, i Ganci, i Brusca, e lui si faceva forza dell’appoggio di questi. Gli hanno lasciato troppo spazio, e lui ci ha rovinato a tutti. Se invece negli anni Sessanta chi lo voleva togliere


{"_id":{"$oid":"5a1c8923eb431bd8a969632f"},"title":["\r\n                    \r\n                        ","\r\n                    \r\n                "," \n                Cecilia Rodriguez non va a Domenica Live, Barbara D’Urso: «Facciamo il 22% anche senza di lei»\n            "," \n                    Cecilia Rodriguez non va a Domenica Live, Barbara D’Urso: «Facciamo il 22% anche senza di lei»\n                "],"text":"","newspaper":"Il Corriere della Sera","resume":["\r\n                    ","\r\n                "],"author":"NP","date":[],"condivisioni":"NP","tags":"NP","link":" http://video.corriere.it/cecilia-rodriguez-non-va-domenica-live-barbara-d-urso-facciamo-22percento-anche-senza-lei/1b6e7d44-d375-11e7-8de2-d9fed093f9f"}

{"_id":{"$oid":"5a1c8923eb431bd8a9696331"},"title":["\r\n                    \r\n                        ","\r\n                    \r\n                "," \n                Un'ora su e giù in ascensore. Reporter deriso da colleghi\n            "," 


{"_id":{"$oid":"5a22823205a983e207e51670"},"title":["Harry e Meghan, forse Natale con la Regina ( Kate era stata esclusa)"],"text":"\n Dopo aver infranto il protocollo reale il giorno dell'annuncio del fidanzamento,  , Harry e Meghan sono pronti a rompere un'altra tradizione:  , sempre ben informato sulle faccende di Buckingham Palace, la futura sposa potrebbe essere invitata a trascorrere il Natale a Sandringham con la Regina e il resto della royal family. Il che sarebbe una novità, visto che nel 2010 a Kate Middleton non fu permesso di unirsi al principe William per i consueti tre giorni di festeggiamenti a Norfolk, sebbene i due ragazzi avessero annunciato il loro fidanzamento un mese prima. Programmi natalizi a parte, l'attenzione mediatica è però tutta concentrata sui dettagli del matrimonio, che verrà celebrato a maggio nel castello di Windsor, e anche se l'attesissima lista degli invitati è ancora tutta da decidere, sempre il tabloid ipotizza che fra gli ospiti potrebbero esser

In [45]:
for i in range(0, len(collection_testi)):
    f= open(actual_journal[0] + actual_journal[1] + "%s.txt" %str(i) , "w+") 
    f.write(collection_testi[i])
    f.close()


In [46]:
# Routine for cleaning documents: CLEAN "NP"
def clean(path):
    with  open(path, 'r') as to_clean:
        one_line = ''
        for string in to_clean.readlines():
            one_line += string.lower()
    to_clean.close()
    
    #Special espressions of html format
    to_replace = ['\\n','\\t','\\r', '\\', '&nbsp'] 
    
    for item in to_replace:
        one_line = one_line.replace(item,' ') 

    cleaned = ' '.join(word for word in one_line.split() if len(word)>1)
    
    # All other special characters
    definitive = re.sub('[^a-zA-Zàéòùè]', ' ', cleaned)
    
    # Just the stopwords remain
    word_tokens = word_tokenize(definitive)
 
    filtered_text = [w for w in word_tokens if not w in stop_words]
    cleaned = ' '.join(word for word in filtered_text if len(word)>1)
    
    with open(path, 'w') as to_clean:
        to_clean.write(cleaned) 
        
    to_clean.close()
    
    return

In [47]:
# Routine for building an interpreter of the stemmed words

def rebuild(parole_intere, parole_stemmizzate):
    vocab = dict()
    
    for i in range(0, len(parole_intere)):
        if parole_stemmizzate[i] not in vocab:
            vocab[parole_stemmizzate[i]] = [parole_intere[i]]
        elif parole_intere[i] not in vocab[parole_stemmizzate[i]]:
            vocab[parole_stemmizzate[i]] =  vocab[parole_stemmizzate[i]] + [parole_intere[i]]
    
    return vocab


In [48]:
nltk.download("stopwords") 
stemmer = SnowballStemmer("italian") 
nltk.download('punkt')
stop_words = set(stopwords.words('italian')) # CHANGE STOPWORDS SET: INCLUDE NP & DOPO & POI & QUANDO & VEDI SOTTO

stemmed_words = []
whole_words = []

for i in range(0, len(collection_testi)):
    file_path = actual_journal[0] + actual_journal[1] + str(i)+ '.txt'
    

    cleaned_text = clean(file_path)

    full_cleaned_text = str("")

    with open(file_path, 'r') as f:
        for line in f.readlines():
            full_cleaned_text = full_cleaned_text + " " + str(line)

    tokens = nltk.word_tokenize(full_cleaned_text)
    whole_words = whole_words + tokens
    stemmed_text = ""

    for token in tokens:
        stemmed_text = stemmed_text + " " + stemmer.stem(token)
        stemmed_words.append(stemmer.stem(token))
    with open(file_path, 'w') as f:
        f.write(stemmed_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [49]:
interpreter = rebuild(whole_words, stemmed_words)


In [50]:
print(interpreter)

{'matte': ['matteo'], 'figl': ['figlio', 'figli', 'figlia', 'figlie'], 'fin': ['finito', 'fino', 'finita', 'fin', 'fine', 'finirai', 'finirebbe', 'finisce', 'finiscono', 'finiamo', 'finirà', 'finite', 'finiti', 'finire', 'finiva', 'fini', 'finirono', 'finirebbero', 'finisca', 'finendo', 'finiranno', 'finirei', 'finisci', 'finimmo'], 'acid': ['acido', 'acida', 'acidi'], 'riin': ['riina'], 'dov': ['doveva', 'dovuto', 'dovevamo', 'dovuti', 'dovevo', 'dovete', 'doverne', 'dovevano', 'dovere', 'dovute', 'dovuta', 'dovendo', 'doversi', 'dovermi', 'doverci', 'dovevate', 'doverle', 'doverla'], 'mor': ['morire', 'mor', 'morendo', 'moro', 'morirono', 'morando', 'morirò', 'more', 'moriva', 'mori'], 'anni': ['anni'], 'fa': ['fa'], 'rom': ['roma', 'romano', 'rom', 'rome'], 'mort': ['morto', 'morte', 'morti', 'morta'], 'tropp': ['troppo', 'troppa', 'troppe', 'troppi'], 'tard': ['tardi', 'tardo', 'tarda', 'tardiva', 'tardivo'], 'cinquant': ['cinquant', 'cinquanta'], 'mormor': ['mormora', 'mormorare']

In [28]:
import findspark
findspark.init("/usr/local/spark") # **
import pyspark
from pyspark import SparkContext
sc = SparkContext()
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)

In [29]:
from pyspark.sql import SQLContext, Row
from pyspark.ml.feature import CountVectorizer
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.ml.feature import StopWordsRemover

lista_topic = []
lista_pesi = []

#------ STARTING LDA PROCEDURE ----------
start_LDA_time = time.time()

for news in range(0, len(collection_testi)): 
    
    file_path = actual_journal[0] + actual_journal[1] + str(news)+ '.txt' 

    data = sc.textFile(file_path).zipWithIndex().map( lambda cleanwords_idd: Row(idd = cleanwords_idd[1], cleanwords = cleanwords_idd[0].split(" ")))
    docDF = spark.createDataFrame(data)


    Vector = CountVectorizer(inputCol="cleanwords", outputCol="vectors")
    model = Vector.fit(docDF)
    result = model.transform(docDF)

    corpus = result.select("idd", "vectors").rdd.map(lambda x_y: [x_y[0],Vectors.fromML(x_y[1])]).cache()

    # Cluster the documents into k topics using LDA
    ldaModel = LDA.train(corpus, k = 2, maxIterations = 100,optimizer='online')
    topics = ldaModel.topicsMatrix()
    vocabArray = model.vocabulary

    weight = ldaModel.describeTopics(maxTermsPerTopic = 10)

    wordNumbers = 10  # number of words per topic
    topicIndices = sc.parallelize(ldaModel.describeTopics(maxTermsPerTopic = wordNumbers))
    
    def topic_render(topic):  # specify vector id of words to actual words
        terms = topic[0]
        result = []
        for i in range(wordNumbers):
            term = vocabArray[terms[i]]
            result.append(term)
        return result
    
    
    topics_final = []
    topics_final = topicIndices.map(lambda topic: topic_render(topic)).collect()
    
    i = 0 
    print('---------- NEWS n° %s' %news)    
    for topic in range(len(topics_final)):
        print ("Topic" + str(topic) + ":")
        j = 0
        terms = ''
        for term in topics_final[topic]:
            print (term + '  ' + str(weight[i][1][j]))
            terms = terms + ' ' + term
            lista_pesi.append(weight[i][1][j])
            j += 1
        i += 1
        print ('\n')
        lista_topic.append(terms) # Qui potrebbe esserci qualcosa che non va... gli devo passare le cose in modo da avere 
                                  # confronti con i documenti...
        
        
print("--- The LDA procedure took  %s seconds  ---" % (time.time() - start_LDA_time))

---------- NEWS n° 0
Topic0:
furb  0.005931063234222752
anch  0.005914402816940607
parl  0.0058897991090801185
innocent  0.005873311806410233
ce  0.005756620063340486
prim  0.005749784994457748
chiud  0.0056742249064629545
sap  0.005635251203808385
fatt  0.0056331874418559975
perd  0.005607561198928655


Topic1:
riin  0.008584870818193643
cos  0.007727252025345316
mort  0.007472557288605044
dov  0.007156018559551024
ammazz  0.007151389469768935
fatt  0.0071215520692192716
strag  0.007011988787192956
maf  0.0067599238748291554
stat  0.006620533518937622
altri  0.006520650383385631


---------- NEWS n° 1
Topic0:
imam  0.008079758620096673
rambl  0.006816044377944377
barcellon  0.00663194943721709
stat  0.006505956874283944
centr  0.006379412426603019
mort  0.00606421346613487
serviz  0.006024621728998821
rapport  0.005943115656893352
terror  0.005889850770717622
quand  0.005887305099134248


Topic1:
rambl  0.0067322050849095835
part  0.0060099914369299455
tratt  0.005883609357432175
amme

---------- NEWS n° 14
Topic0:
scors  0.007934379609132325
fin  0.00789134421495005
possibil  0.007524733131239366
studentess  0.0075196979903843116
uom  0.007442372038293415
stat  0.007441437534470313
pen  0.0071140768394469004
disoccup  0.007090197857700609
proced  0.007040309468548941
enne  0.0068550032292815566


Topic1:
propr  0.007481184091173422
enne  0.007352172169132689
insult  0.0072834103318906345
spatar  0.007163979854535311
venerd  0.007146957949620719
luned  0.007054703111057882
conness  0.00702851307521914
disoccup  0.007024653548066149
relat  0.007000229246049729
studentess  0.006994337697891124


---------- NEWS n° 15
Topic0:
contratt  0.006447110029352394
apprend  0.006354277354215251
res  0.0063431915051219735
gioved  0.006252408644711305
capital  0.006036915037882267
indispens  0.006004936809438093
raggiunt  0.005958311973762771
genoves  0.005957901434665005
dipendent  0.005933209304136593
sufficient  0.005925807358068392


Topic1:
banc  0.009144948331137359
carig  0

---------- NEWS n° 28
Topic0:
certos  0.023152928668437767
  0.02229281877528248
disposizion  0.021312770164094195
dop  0.021239351174254014
domicil  0.021170352124158916
provved  0.02094166391320385
croc  0.02088430467430531
addi  0.020704089498681695
mes  0.020658979561769767
new  0.020369787213760638


Topic1:
veron  0.02261968345079352
giudic  0.022154807749062276
new  0.021736112201611847
anno  0.021735459929162582
york  0.021023121045920804
np  0.02087105136788934
lar  0.02083004235332937
ora  0.02077691424564299
cas  0.020644076536759558
  0.020438817723616852


---------- NEWS n° 29
Topic0:
scors  0.010603543384850403
prisc  0.010427404938872214
uom  0.0104197690305486
fint  0.010309145605906058
aggressor  0.010300675566589186
canades  0.010281253164212462
essers  0.010207797641478066
manc  0.0100480650355728
dar  0.010005069265794597
piemontes  0.00994127689239423


Topic1:
mil  0.011106418748112962
stat  0.010875199388992871
gip  0.010737389172613803
scont  0.0106828554035002

---------- NEWS n° 42
Topic0:
fall  0.006055619709200077
rowling  0.005375780696408073
cos  0.005303168808493651
discors  0.0047738546173668305
esser  0.0047438165984195636
vit  0.004391895114603433
divent  0.004294965326782189
sugger  0.004286330462779718
giovan  0.00426682073902778
laur  0.0041584451028713965


Topic1:
viss  0.003991675880527188
fin  0.003990065850958354
psicolog  0.003951930860148009
figur  0.003937890774263591
realizz  0.0038963801449454147
elabor  0.0038720202933259663
teor  0.0038461777536777523
grand  0.0038355148927710294
chiunqu  0.0038198266920422405
capac  0.003808000938008693


---------- NEWS n° 43
Topic0:
casc  0.014936903680365854
agu  0.013733998273572967
azul  0.013584282291241534
scorr  0.011587190661433024
acqua  0.01129370085589527
campesinos  0.011016069329600834
turches  0.010866988911097185
salv  0.010848819540305093
nuov  0.010779182474058614
natural  0.010733231401341536


Topic1:
secc  0.011748621809152404
alve  0.011204122383872599
dett  0.01

---------- NEWS n° 56
Topic0:
pan  0.02587815663811535
mens  0.02573243509526099
chiod  0.021275472760163696
alcun  0.02112566201738987
commission  0.020907255858819158
piccol  0.01969151337676316
trov  0.01941079405239935
scuol  0.01934249089023899
dane  0.01918541969259117
element  0.019017693646902107


Topic1:
local  0.021551966596607804
commission  0.020231086881259693
mess  0.020085404384357512
quint  0.019900100006278808
rifer  0.019741869466332537
pranz  0.01955341477972945
poliz  0.019369018476992975
avvis  0.01933154297777856
pezz  0.019306979415788985
element  0.01927464246847706


---------- NEWS n° 57
Topic0:
amor  0.007101776041321423
cuor  0.006137209753075983
trov  0.006088813001877837
propr  0.0059438297186565035
com  0.005913581922210929
appag  0.005854491664129497
complic  0.005854187787150192
cresc  0.005792370817234466
scelt  0.0057347261194702
andrebb  0.005697431280250918


Topic1:
amor  0.015715526122013806
sent  0.008373295465402362
vit  0.007829234203766805
co

...

---------- NEWS n° 672
Topic0:
elin  0.01248386698277206
atlet  0.011196231372020853
stat  0.008842226219237859
torne  0.00859953496468122
ucrain  0.00854234460848605
internazional  0.008535616038019776
carrier  0.008519062761119247
sol  0.008426920739606634
pos  0.008126896932944324
mes  0.008110137439086172


Topic1:
sexy  0.008121120243187481
fan  0.007989934070133693
osserv  0.007850583803597466
camp  0.00762893504924087
protagon  0.0076158514757854795
posizion  0.007574309493786981
fier  0.007552339651974993
qualc  0.007441839695170066
russ  0.007435822948179855
torne  0.007378985732169444


---------- NEWS n° 673
Topic0:
chiellin  0.00974684420736279
capit  0.009291734127927595
problem  0.009209252068746966
ben  0.00901416250158043
grec  0.008902291082103098
super  0.008894490114072335
ennesim  0.008846855372141912
invec  0.008812390167871517
vint  0.00856426574560817
matc  0.008542718763307209


Topic1:
ritrov  0.009537951057600072
ben  0.00940628833725685
allen  0.0092851688610

---------- NEWS n° 686
Topic0:
par  0.004558082863578771
davver  0.004537936211060383
med  0.004518706851970872
sci  0.004518015181056092
amichett  0.004392919295354139
trascur  0.004387131096136355
oper  0.004375757698131456
me  0.004363348110999254
struttur  0.0043583874637445445
primar  0.004259685393264119


Topic1:
qualcun  0.004799810074003704
difficil  0.004499998075360035
vior  0.004397611479366969
settembr  0.004375862116824675
cent  0.0043549377784626786
allegr  0.004330531547969948
ven  0.004284168837917485
fatt  0.00426299345504162
tant  0.004251614602016404
gioc  0.004236361318698604


---------- NEWS n° 687
Topic0:
po  0.005224639009438042
insign  0.005222648527653444
perpless  0.005123717895242216
mer  0.0050797424743414385
sarr  0.005077444728755781
qui  0.005058068484697018
tunnel  0.0050476334224979396
nerv  0.004975815834813966
belg  0.004965558022926148
far  0.0049409494080055565


Topic1:
napol  0.00750113602541594
temp  0.005926134573246846
prim  0.005725650651605

---------- NEWS n° 700
Topic0:
comun  0.00733111063912838
dop  0.007142877740356954
esser  0.006116420397744876
spieg  0.006092821722477648
cos  0.006026036617174432
lavor  0.005942885418908545
andre  0.005925517964233962
tutor  0.005923409923223393
graz  0.005812882156847293
fond  0.005652466346846207


Topic1:
rappresent  0.005683815533565298
andavam  0.005590816962530173
tre  0.005571617439647014
ce  0.005565416349725533
lav  0.0055547132228370285
multinazional  0.005530474403629487
svit  0.0055069659750012845
senz  0.005505099326635196
comun  0.005457980171274405
graz  0.005404706375493782


---------- NEWS n° 701
Topic0:
scienz  0.006368167584008859
crucial  0.006219644871360571
mod  0.00618412226828621
sempr  0.006161100872250561
general  0.006130479978162126
zon  0.006126697790318485
capac  0.0061077510325191755
canad  0.006026584197779601
ital  0.005998058640615427
esatt  0.0059201472204401745


Topic1:
class  0.012317888430984685
student  0.010266416994247685
immigr  0.0096319

---------- NEWS n° 714
Topic0:
ospedal  0.01459855619309627
avor  0.014572895000868245
manett  0.014345121380267273
timor  0.014013904264532558
butt  0.013959503611027739
cas  0.013771594392525574
terr  0.01362580121339765
arriv  0.013370902695666625
carabinier  0.013252542636411619
prim  0.013240488306129262


Topic1:
donn  0.01600052113767091
ferm  0.01433332775838596
giovan  0.01432499311128999
person  0.014309870983507687
poi  0.014104296774550789
stat  0.014086159725601089
anni  0.013777215957221641
blocc  0.01360598464879541
quart  0.013561236444115575
grav  0.013375231810478965


---------- NEWS n° 715
Topic0:
lavor  0.009915448767598145
azi  0.009795745839366399
vend  0.009580925675873224
pandor  0.009575535270121548
prossim  0.008907682317239166
melegatt  0.008899887507078314
fin  0.008574114746043275
pezz  0.0083785868196553
lasc  0.008372024031288533
cass  0.008209956614952189


Topic1:
super  0.008588406246553102
dimostr  0.008334806673499853
pien  0.008221136708228214
marc

---------- NEWS n° 728
Topic0:
stral  0.00487701718347069
artist  0.0047429981278295985
client  0.004742361432567833
catt  0.004705631416836953
spars  0.004686990561008904
ateism  0.004682918874696261
nud  0.0046418527574183235
fas  0.0046317754211398655
quell  0.0046292685254793395
legger  0.004620314827384091


Topic1:
cos  0.008080114410184566
balthus  0.00576175709586082
neo  0.005753050611718617
men  0.005597304172132014
fors  0.005298336081869982
centenar  0.00517923672646095
due  0.005116360917005655
new  0.005026889885433957
thèrès  0.005000639788999647
inquiet  0.004950312473405143


---------- NEWS n° 729
Topic0:
molt  0.023832839484602637
sci  0.023402531628603902
mentr  0.02314442431041953
lavor  0.02229876738910799
flessor  0.022142684207585292
corsett  0.02172859452875953
allen  0.02149856445364124
infin  0.021387592098472866
lombar  0.02079430023948999
muscol  0.02038599522331154


Topic1:
flessor  0.021684337725836444
possibil  0.02159685796615832
pav  0.021352521760176

In [30]:
print(lista_topic)

[' furb anch parl innocent ce prim chiud sap fatt perd', ' riin cos mort dov ammazz fatt strag maf stat altri', ' imam rambl barcellon stat centr mort serviz rapport terror quand', ' rambl part tratt ammett person ora travolgess prim cerc on', ' nuov prenot tesl produtt comunqu elettr almen tip confin batter', ' fresc attend svel assicur elon distrarr problem nessun fors present', ' povert torn rispos cos scatt louis mentr donald que the', ' ner louis pell sol linton cas fot vest web mnuchin', ' unic charleston fa mod vital fantas tendenz poss fior francesc', ' individual nient espert sneaker aggiung simil fantas mess modernissim divert', ' inter gol autunnal der rest classif med allung juventus benevent', ' rom favor classif curios vicin post gol sorpres grand part', ' parol raccont fest lettur propr autor sal novembr stor doman', ' lettur parol numer novembr fest autor raccont bookcity artist sett', ' scuol figl genitor adess student qui croll comand occup carl', ' comunqu gliel scor

In [55]:
vectorizer = TfidfVectorizer() # use_idf=True, ngram_range=(1, 3)
 
#------ STARTING LSI PROCEDURE ----------
start_LSI_time = time.time()

X = vectorizer.fit_transform(lista_topic) 

lsa = TruncatedSVD(n_components = 1, n_iter = 1000) # Gauge the components
lsa.fit(X)

terms = vectorizer.get_feature_names()
out = []
for i, comp in enumerate(lsa.components_):
    termsInComp = zip(terms,comp)
    sortedTerms = sorted(termsInComp, key=lambda x: x[1], reverse=True ) [:100]
    
    for term in sortedTerms:
        out.append(term[0])
        
print("--- The LSI procedure took  %s seconds  ---" % (time.time() - start_LSI_time))

# We could sum and/or compare times at this point...

--- The LSI procedure took  2.7752113342285156 seconds  ---


In [56]:
# Use the interpreter
# print(out)
for w in out:
    print(w, interpreter[w])

stat ['stato', 'stati', 'stata', 'state', 'statuto']
anni ['anni']
cos ['cosa', 'cos', 'cose', 'cosio', 'cosi']
prim ['prime', 'prima', 'primo', 'primi', 'primato', 'primati', 'prim']
sol ['solo', 'sole', 'solare', 'sola', 'solito', 'soli', 'sol', 'soliti', 'solamente', 'solita', 'solite']
dop ['dopo', 'dop']
quand ['quando', 'quand']
part ['parte', 'partita', 'parto', 'partire', 'parti', 'partite', 'partito', 'partirebbe', 'partiti', 'partono', 'partirà', 'partendo', 'partiva', 'parta', 'partirono', 'partivano', 'partano']
esser ['essere', 'esseri', 'esser']
lavor ['lavoro', 'lavori', 'lavorando', 'lavorato', 'lavoratori', 'lavora', 'lavorativo', 'lavorative', 'lavorare', 'lavorava', 'lavoratore', 'lavoriamo', 'lavorano', 'lavoravano', 'lavorerà', 'lavoravo', 'lavorativa', 'lavoratrice', 'lavorati', 'lavoratrici', 'lavorazione', 'lavorò']
ital ['italia', 'italiano', 'italo', 'italiche', 'italici', 'italico']
mil ['milano', 'mila', 'milita']
np ['np']
donn ['donna', 'donne']
cas ['case

In [107]:
#  Saving
with open(path + actual_journal[1] + ' Output', "w") as output:
    output.write("\n".join(map(lambda x: str(x), out)))